## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import keras_tuner as kt
import numpy as np

In [2]:
# read the loan data

loan_df = pd.read_csv("loan-train.csv")
loan_df.head()



,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [3]:
loan_cleaned_df = loan_df.dropna(axis=0, how='any')
loan_cleaned_df.head()
loan_cleaned_df.shape


(480, 13)

In [4]:
# Drop the non-beneficial ID columns, 'Loan_ID' 
loan_cleaned_df = loan_cleaned_df.drop(columns=['Loan_ID'])
loan_cleaned_df.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
1,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
5,Male,Yes,2,Graduate,Yes,5417,4196.0,267.0,360.0,1.0,Urban,Y


In [5]:
# Determine the number of unique values in each column.
loan_cleaned_df.nunique()

Gender                 2
Married                2
Dependents             4
Education              2
Self_Employed          2
ApplicantIncome      405
CoapplicantIncome    232
LoanAmount           186
Loan_Amount_Term       9
Credit_History         2
Property_Area          3
Loan_Status            2
dtype: int64

In [6]:
# Determine the number of unique values in each column.
loan_cleaned_df.dtypes

Gender                object
Married               object
Dependents            object
Education             object
Self_Employed         object
ApplicantIncome        int64
CoapplicantIncome    float64
LoanAmount           float64
Loan_Amount_Term     float64
Credit_History       float64
Property_Area         object
Loan_Status           object
dtype: object

In [7]:
# Convert categorical data to numeric with `pd.get_dummies`

loan_df_numeric = pd.get_dummies(loan_cleaned_df, 
                            columns=['Gender','Married','Dependents',
                                     'Education','Self_Employed','Property_Area',
                                     'Loan_Status'], dtype=int)

loan_df_numeric.head()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Gender_Female,Gender_Male,Married_No,Married_Yes,Dependents_0,...,Dependents_3+,Education_Graduate,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban,Loan_Status_N,Loan_Status_Y
1,4583,1508.0,128.0,360.0,1.0,0,1,0,1,0,...,0,1,0,1,0,1,0,0,1,0
2,3000,0.0,66.0,360.0,1.0,0,1,0,1,1,...,0,1,0,0,1,0,0,1,0,1
3,2583,2358.0,120.0,360.0,1.0,0,1,0,1,1,...,0,0,1,1,0,0,0,1,0,1
4,6000,0.0,141.0,360.0,1.0,0,1,1,0,1,...,0,1,0,1,0,0,0,1,0,1
5,5417,4196.0,267.0,360.0,1.0,0,1,0,1,0,...,0,1,0,0,1,0,0,1,0,1


In [8]:
loan_df_numeric.shape

(480, 22)

In [9]:
loan_df_numeric.rename(columns = {'Loan_Status_Y':'Loan_Status'}, inplace = True)
loan_df_numeric=loan_df_numeric.drop(columns='Loan_Status_N')

loan_df_numeric.head()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Gender_Female,Gender_Male,Married_No,Married_Yes,Dependents_0,...,Dependents_2,Dependents_3+,Education_Graduate,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban,Loan_Status
1,4583,1508.0,128.0,360.0,1.0,0,1,0,1,0,...,0,0,1,0,1,0,1,0,0,0
2,3000,0.0,66.0,360.0,1.0,0,1,0,1,1,...,0,0,1,0,0,1,0,0,1,1
3,2583,2358.0,120.0,360.0,1.0,0,1,0,1,1,...,0,0,0,1,1,0,0,0,1,1
4,6000,0.0,141.0,360.0,1.0,0,1,1,0,1,...,0,0,1,0,1,0,0,0,1,1
5,5417,4196.0,267.0,360.0,1.0,0,1,0,1,0,...,1,0,1,0,0,1,0,0,1,1


In [10]:
# Split our preprocessed data into our features and target arrays
y = loan_df_numeric['Loan_Status'].values
X = loan_df_numeric.drop(columns='Loan_Status').values
print(X.shape)

(480, 20)


In [11]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [12]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [45]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.


nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation="elu", input_dim=20))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=30, activation="elu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                1680      
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 4,141
Trainable params: 4,141
Non-trainable params: 0
_________________________________________________________________


In [46]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [47]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
12/12 [==============================] - 1s 3ms/step - loss: 0.6586 - accuracy: 0.6333
Epoch 2/100
12/12 [==============================] - 0s 2ms/step - loss: 0.5377 - accuracy: 0.7639
Epoch 3/100
12/12 [==============================] - 0s 2ms/step - loss: 0.4877 - accuracy: 0.8000
Epoch 4/100
12/12 [==============================] - 0s 4ms/step - loss: 0.4579 - accuracy: 0.8194
Epoch 5/100
12/12 [==============================] - 0s 4ms/step - loss: 0.4522 - accuracy: 0.8194
Epoch 6/100
12/12 [==============================] - 0s 3ms/step - loss: 0.4451 - accuracy: 0.8194
Epoch 7/100
12/12 [==============================] - 0s 3ms/step - loss: 0.4380 - accuracy: 0.8250
Epoch 8/100
12/12 [==============================] - 0s 3ms/step - loss: 0.4376 - accuracy: 0.8278
Epoch 9/100
12/12 [==============================] - 0s 3ms/step - loss: 0.4347 - accuracy: 0.8306
Epoch 10/100
12/12 [==============================] - 0s 4ms/step - loss: 0.4344 - accuracy: 0.8222
Epoch 11/

12/12 [==============================] - 0s 4ms/step - loss: 0.2833 - accuracy: 0.8917
Epoch 84/100
12/12 [==============================] - 0s 4ms/step - loss: 0.2862 - accuracy: 0.8833
Epoch 85/100
12/12 [==============================] - 0s 3ms/step - loss: 0.2868 - accuracy: 0.8806
Epoch 86/100
12/12 [==============================] - 0s 3ms/step - loss: 0.2811 - accuracy: 0.8861
Epoch 87/100
12/12 [==============================] - 0s 3ms/step - loss: 0.2825 - accuracy: 0.8889
Epoch 88/100
12/12 [==============================] - 0s 3ms/step - loss: 0.2821 - accuracy: 0.8889
Epoch 89/100
12/12 [==============================] - 0s 3ms/step - loss: 0.2758 - accuracy: 0.8972
Epoch 90/100
12/12 [==============================] - 0s 3ms/step - loss: 0.2795 - accuracy: 0.8917
Epoch 91/100
12/12 [==============================] - 0s 3ms/step - loss: 0.2742 - accuracy: 0.9000
Epoch 92/100
12/12 [==============================] - 0s 3ms/step - loss: 0.2737 - accuracy: 0.8889
Epoch 93/100


In [48]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

4/4 - 0s - loss: 0.6267 - accuracy: 0.7500 - 257ms/epoch - 64ms/step
Loss: 0.6267025470733643, Accuracy: 0.75


In [50]:
#Try Keras-tuner to improve the model

# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_2 = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','elu','tanh'])
    optimizer = hp.Choice('optimizer',['SGD','Adam'])
    
    # Allow kerastuner to decide number of neurons in first layer
    nn_2.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=80,
        step=5), activation=activation, input_dim=20))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 10)):
        nn_2.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=80,
            step=5),
            activation=activation))
    
    nn_2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_2.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])
    
    return nn_2


In [51]:
tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=50,
    hyperband_iterations=2)

In [52]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=50,validation_data=(X_test_scaled,y_test))

Trial 180 Complete [00h 00m 11s]
val_accuracy: 0.7666666507720947

Best val_accuracy So Far: 0.8083333373069763
Total elapsed time: 00h 14m 54s
INFO:tensorflow:Oracle triggered exit


In [13]:
#look for outliers
#start with applicant income

app_income = loan_df_numeric['ApplicantIncome']

quartiles = np.quantile(app_income,[.25,.75])
iqr = quartiles[1]-quartiles[0]
lower_bound = quartiles[0]-(1.5*iqr)
upper_bound = quartiles[1]+(1.5*iqr)
print (f"quartiles = {quartiles} & IQR = {iqr}")
print (f"lower bound = {lower_bound} & upper bound = {upper_bound}")

potential_outliers_app_income = [value for value in app_income if value < lower_bound or value > upper_bound]
potential_outliers_app_income

quartiles = [2898.75 5852.5 ] & IQR = 2953.75
lower bound = -1531.875 & upper bound = 10283.125


[12841,
 12500,
 11500,
 10750,
 11417,
 14583,
 10408,
 10513,
 14999,
 11757,
 14866,
 39999,
 33846,
 39147,
 12000,
 11000,
 16250,
 14683,
 11146,
 14583,
 20233,
 15000,
 19730,
 15759,
 81000,
 14880,
 10416,
 37719,
 16525,
 16667,
 10833,
 18333,
 17263,
 13262,
 17500,
 18165,
 19484,
 16666,
 16120,
 12000]

In [14]:
# Delete Rows with the Applicant Income outliers

loan_new1 = loan_df_numeric.drop(loan_df_numeric[loan_df_numeric['ApplicantIncome'] > 10283.125].index)

In [15]:
#look for outliers
#coapplicant income

co_app_income = loan_new1['CoapplicantIncome']

quartiles = np.quantile(co_app_income,[.25,.75])
iqr = quartiles[1]-quartiles[0]
lower_bound = quartiles[0]-(1.5*iqr)
upper_bound = quartiles[1]+(1.5*iqr)
print (f"quartiles = {quartiles} & IQR = {iqr}")
print (f"lower bound = {lower_bound} & upper bound = {upper_bound}")

potential_outliers_co_app_income = [value for value in co_app_income if value < lower_bound or value > upper_bound]
potential_outliers_co_app_income

quartiles = [   0.   2312.75] & IQR = 2312.75
lower bound = -3469.125 & upper bound = 5781.875


[8106.0,
 7210.0,
 8980.0,
 7750.0,
 11300.0,
 7250.0,
 7101.0,
 6250.0,
 7873.0,
 20000.0,
 20000.0,
 6666.0,
 7166.0,
 33837.0]

In [16]:
# Delete Rows with the Co-Applicant Income outliers

loan_new2 = loan_new1.drop(loan_new1[loan_new1['CoapplicantIncome'] > 5781.875].index)


In [17]:
# Look at Loan term value counts to potentially consolidate into bins

loan_new2['Loan_Amount_Term'].value_counts()

Loan_Amount_Term
360.0    364
180.0     30
480.0     12
300.0      8
120.0      3
84.0       3
60.0       2
240.0      2
36.0       2
Name: count, dtype: int64

In [18]:
# Consolidate all loans that are 120, 84, 60, 240, and 36 into an "other" group

# create list of loans to consolidate
term_consolidation = [120, 84, 60, 240, 36]

# Replace in dataframe
for term in term_consolidation:
    loan_new2['Loan_Amount_Term'] = loan_new2['Loan_Amount_Term'].replace(term,"Other")

# Check value_counts again
loan_new2['Loan_Amount_Term'].value_counts()

Loan_Amount_Term
360.0    364
180.0     30
Other     12
480.0     12
300.0      8
Name: count, dtype: int64

In [22]:
# Convert term data to numeric with `pd.get_dummies`

loan_new2_numeric = pd.get_dummies(loan_new2, 
                            columns=['Loan_Amount_Term'], dtype=int)

loan_new2_numeric.head()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Credit_History,Gender_Female,Gender_Male,Married_No,Married_Yes,Dependents_0,Dependents_1,...,Self_Employed_Yes,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban,Loan_Status,Loan_Amount_Term_180.0,Loan_Amount_Term_300.0,Loan_Amount_Term_360.0,Loan_Amount_Term_480.0,Loan_Amount_Term_Other
1,4583,1508.0,128.0,1.0,0,1,0,1,0,1,...,0,1,0,0,0,0,0,1,0,0
2,3000,0.0,66.0,1.0,0,1,0,1,1,0,...,1,0,0,1,1,0,0,1,0,0
3,2583,2358.0,120.0,1.0,0,1,0,1,1,0,...,0,0,0,1,1,0,0,1,0,0
4,6000,0.0,141.0,1.0,0,1,1,0,1,0,...,0,0,0,1,1,0,0,1,0,0
5,5417,4196.0,267.0,1.0,0,1,0,1,0,0,...,1,0,0,1,1,0,0,1,0,0


In [37]:
# Split our preprocessed data into our features and target arrays
y2 = loan_new2_numeric['Loan_Status'].values
X2 = loan_new2_numeric.drop(columns='Loan_Status').values
print(X2.shape)

(426, 24)


In [38]:
# Split the preprocessed data into a training and testing dataset
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, random_state=42)

In [39]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X2_scaler = scaler.fit(X2_train)

# Scale the data
X2_train_scaled = X2_scaler.transform(X2_train)
X2_test_scaled = X2_scaler.transform(X2_test)

In [40]:
#Try Keras-tuner to improve the model

# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_2 = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','elu','tanh'])
    optimizer = hp.Choice('optimizer',['SGD','Adam'])
    
    # Allow kerastuner to decide number of neurons in first layer
    nn_2.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=80,
        step=5), activation=activation, input_dim=24))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 5)):
        nn_2.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=80,
            step=5),
            activation=activation))
    
    nn_2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_2.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])
    
    return nn_2


In [41]:
tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=50,
    hyperband_iterations=2)

INFO:tensorflow:Reloading Tuner from .\untitled_project\tuner0.json


In [42]:
# Run the kerastuner search for best hyperparameters
tuner.search(X2_train_scaled,y2_train,epochs=50,validation_data=(X2_test_scaled,y2_test))

Trial 174 Complete [00h 00m 08s]
val_accuracy: 0.7850467562675476

Best val_accuracy So Far: 0.822429895401001
Total elapsed time: 00h 12m 37s
INFO:tensorflow:Oracle triggered exit
